<a href="https://colab.research.google.com/github/stevejj4/Customer-Reward-Analysis/blob/main/Offers_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import auth #Authorizing google colab
auth.authenticate_user()
import gspread #credentials for google sheet
from google.auth import default
creds,_ = default()
gc = gspread.authorize(creds) #Authorizing the connection
import pandas as pd
worksheet = gc.open('Offers').sheet1 #defining the worksheet
rows = worksheet.get_all_values() #getting all values in list of row
df = pd.DataFrame(rows)
df.columns = df.iloc[0] #creating columns name
df = df.iloc[1:]

In [40]:
df.head()

,offer_id,offer_type,difficulty,reward,duration,channels
1,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
2,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
3,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
5,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"


In [41]:
# total number of offers used
print("The total number of offers used are : ", df.shape[0])

The total number of offers used are :  10


In [42]:
# Converting the strings representing lists in the channels column to actual lists
import ast
df['channels'] = df['channels'].apply(ast.literal_eval)

In [45]:
# One hot encoding
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_channels = pd.DataFrame(mlb.fit_transform(df['channels']),columns=mlb.classes_)
df = pd.concat([df, df_channels], axis=1)
df = df.drop('channels', axis=1)

In [46]:
df.head(10)

,offer_id,offer_type,difficulty,reward,duration,email,mobile,social,web
1,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,1.0,1.0,1.0,1.0
2,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,1.0,1.0,0.0,1.0
3,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,1.0,1.0,0.0,1.0
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,1.0,0.0,0.0,1.0
5,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,1.0,1.0,1.0,1.0
6,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7,1.0,1.0,1.0,1.0
7,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,1.0,1.0,1.0,0.0
8,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,1.0,1.0,1.0,1.0
9,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,1.0,1.0,0.0,1.0
10,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,NaN,NaN,NaN,NaN


In [49]:
# Convert 'difficulty' and 'reward' to numeric, coercing errors
df['difficulty'] = pd.to_numeric(df['difficulty'], errors='coerce')
df['reward'] = pd.to_numeric(df['reward'], errors='coerce')

# Grouping by offer_type and calculating mean difficulty and reward
offer_group = df.groupby('offer_type')[['difficulty', 'reward']].mean()

# Display the transformed DataFrame

offer_group


,difficulty,reward
offer_type,,
bogo,7.50,7.5
discount,11.75,3.0
informational,0.00,0.0
